In [2]:
import numpy as np
import pandas as pd
from numpy import log2 as log

In [3]:
eps = np.finfo(float).eps

In [4]:
dataset = {
'Id':[1,2,3,4,5,6,7,8,9,10,11,12,13,14],
'Age':['<21','<21','21-35','>35','>35','>35','21-35','<21','<21','>35','<21','21-35','21-35','>35'],
'Income':['High','High','High','Medium','Low','Low','Low','Medium','Low','Medium','Medium','Medium','High','Medium'],
'Gender':['Male','Male','Male','Male','Female','Female','Female','Male','Female','Female','Female','Male','Female','Male'],
'MaritalStatus':['Single','Married','Single','Single','Single','Married','Married','Single','Married','Single','Married','Married','Single','Married'],
'Buys':['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']}


In [5]:
df = pd.DataFrame(dataset,columns=['ID', 'Age', 'Income', 'Gender', 'MaritalStatus', 'Buys' ])

In [6]:
df

,ID,Age,Income,Gender,MaritalStatus,Buys
0,NaN,<21,High,Male,Single,No
1,NaN,<21,High,Male,Married,No
2,NaN,21-35,High,Male,Single,Yes
3,NaN,>35,Medium,Male,Single,Yes
4,NaN,>35,Low,Female,Single,Yes
5,NaN,>35,Low,Female,Married,No
6,NaN,21-35,Low,Female,Married,Yes
7,NaN,<21,Medium,Male,Single,No
8,NaN,<21,Low,Female,Married,Yes
9,NaN,>35,Medium,Female,Single,Yes


In [7]:
df = df.drop('ID', axis=1)
df

,Age,Income,Gender,MaritalStatus,Buys
0,<21,High,Male,Single,No
1,<21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,>35,Medium,Male,Single,Yes
4,>35,Low,Female,Single,Yes
5,>35,Low,Female,Married,No
6,21-35,Low,Female,Married,Yes
7,<21,Medium,Male,Single,No
8,<21,Low,Female,Married,Yes
9,>35,Medium,Female,Single,Yes


In [8]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy

In [9]:
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   
    target_variables = df[Class].unique()
    variables = df[attribute].unique()  
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

In [10]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
#Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [11]:
def get_subtable(df, node, value):
  return df[df[node] == value].reset_index(drop=True)

In [12]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]
    node = find_winner(df)
    attValue = np.unique(df[node])
    if tree is None:                    
        tree={}
        tree[node] = {}

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Buys'],return_counts=True)                        
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 
                   
    return tree

In [13]:
tree = buildTree(df)

In [14]:
tree

{'Age': {'21-35': 'Yes',
  '<21': {'Gender': {'Female': 'Yes', 'Male': 'No'}},
  '>35': {'MaritalStatus': {'Married': 'No', 'Single': 'Yes'}}}}

In [15]:
def predict(inst, tree):
    for nodes in tree.keys():
      value = inst[nodes]
      tree = tree[nodes][value]
      prediction = 0

      if type(tree) is dict:
        prediction = predict(inst, tree)
      else:
        prediction = tree
        break;
      
      return prediction

In [16]:
inst = df.iloc[6]

In [17]:
inst

Age                21-35
Income               Low
Gender            Female
MaritalStatus    Married
Buys                 Yes
Name: 6, dtype: object

In [19]:
Prediction = predict(inst, tree)

In [20]:
Prediction

In [21]:
#Test on new Data
data = {'Age':'<21','Income':'Low','Gender':'Female','MaritalStatus':'Married'}

In [22]:
data

{'Age': '<21', 'Gender': 'Female', 'Income': 'Low', 'MaritalStatus': 'Married'}

In [23]:
NewPrediction = predict(data,tree)

In [24]:
NewPrediction

In [25]:
'Yes'

'Yes'